In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-10 14:08:27.975717: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 14:08:28.076402: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 14:08:28.076416: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 14:08:28.097680: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 14:08:28.632674: W tensorflow/stream_executor/platform/de

In [2]:
name_all=[]
auc_roc=[]
name='all_algorithm'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100


2022-11-10 14:08:32.927883: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 14:08:32.928384: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-10 14:08:32.928432: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-10 14:08:32.928471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-10 14:08:32.928509: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 389ms/step - loss: 2.5677 - auc: 0.4059
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.8663 - auc: 0.8318
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 1.9260 - auc: 0.8751
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.9726 - auc: 0.8881
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.9339 - auc: 0.8941
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.7921 - auc: 0.9061
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6205 - auc: 0.9188
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4676 - auc: 0.9277
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3354 - auc: 0.9414
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2539 - auc: 0.9520
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2518 

1/1 [==============================] - 0s 24ms/step - loss: 0.1869 - auc: 0.9963
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1926 - auc: 0.9948
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1948 - auc: 0.9947
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1898 - auc: 0.9955
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1932 - auc: 0.9938
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1901 - auc: 0.9959
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1916 - auc: 0.9940
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1871 - auc: 0.9954
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1903 - auc: 0.9946
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1806 - auc: 0.9964
Epoch 98/100
1/1 [==============================] - 0s 30ms/step - loss: 

In [3]:
name='remove_dmpnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 332ms/step - loss: 2.1273 - auc: 0.4467
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.5865 - auc: 0.8113
Epoch 3/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6196 - auc: 0.8577
Epoch 4/100
1/1 [==============================] - 0s 26ms/step - loss: 1.6835 - auc: 0.8625
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6331 - auc: 0.8718
Epoch 6/100
1/1 [==============================] - 0s 36ms/step - loss: 1.4941 - auc: 0.8885
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3427 - auc: 0.8950
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1877 - auc: 0.9128
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.1311 - auc: 0.9112
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.0766 - auc: 0.9168
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 28ms/step - loss: 0.2222 - auc: 0.9933
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2282 - auc: 0.9916
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2148 - auc: 0.9946
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2191 - auc: 0.9938
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2222 - auc: 0.9932
Epoch 93/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2223 - auc: 0.9931
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2251 - auc: 0.9927
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2120 - auc: 0.9946
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2115 - auc: 0.9948
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2183 - auc: 0.9934
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [4]:
name='remove_lr'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 332ms/step - loss: 2.3270 - auc: 0.3836
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.5849 - auc: 0.7986
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6355 - auc: 0.8530
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7152 - auc: 0.8696
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 1.7483 - auc: 0.8692
Epoch 6/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6546 - auc: 0.8789
Epoch 7/100
1/1 [==============================] - 0s 36ms/step - loss: 1.5248 - auc: 0.8887
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3664 - auc: 0.8983
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2268 - auc: 0.9081
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.1401 - auc: 0.9132
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2383 - auc: 0.9899
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2366 - auc: 0.9905
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2347 - auc: 0.9916
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2420 - auc: 0.9892
Epoch 92/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2309 - auc: 0.9919
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2390 - auc: 0.9894
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2259 - auc: 0.9926
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2327 - auc: 0.9913
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2233 - auc: 0.9925
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2366 - auc: 0.9904
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [5]:
name='remove_fcnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','lr','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 340ms/step - loss: 1.8934 - auc: 0.6044
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6628 - auc: 0.8331
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6614 - auc: 0.8659
Epoch 4/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6229 - auc: 0.8753
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4556 - auc: 0.8932
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3139 - auc: 0.9017
Epoch 7/100
1/1 [==============================] - 0s 25ms/step - loss: 1.1701 - auc: 0.9257
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1440 - auc: 0.9201
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.0806 - auc: 0.9321
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 1.0219 - auc: 0.9451
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.1909 - auc: 0.9945
Epoch 89/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1843 - auc: 0.9960
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1874 - auc: 0.9955
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1941 - auc: 0.9944
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1825 - auc: 0.9959
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1916 - auc: 0.9943
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1765 - auc: 0.9970
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1841 - auc: 0.9955
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1898 - auc: 0.9955
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1907 - auc: 0.9948
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [6]:
name='remove_xgb'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['fcnn','lr','dmpnn']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 334ms/step - loss: 3.7982 - auc: 0.2236
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - loss: 2.1129 - auc: 0.4793
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 1.5448 - auc: 0.8289
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5950 - auc: 0.8718
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.7553 - auc: 0.8742
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.8574 - auc: 0.8767
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.8929 - auc: 0.8730
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.9031 - auc: 0.8780
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 1.8391 - auc: 0.8800
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.6943 - auc: 0.8990
Epoch 11/100
1/1 [==============================] - 0s 25ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2740 - auc: 0.9892
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2820 - auc: 0.9875
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2792 - auc: 0.9877
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2696 - auc: 0.9892
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2715 - auc: 0.9889
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2684 - auc: 0.9893
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2844 - auc: 0.9856
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2750 - auc: 0.9876
Epoch 96/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2739 - auc: 0.9873
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2681 - auc: 0.9881
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [7]:
de=pd.DataFrame()
de['name']=name_all
de['auc_roc']=auc_roc
de.to_csv('4model_remove_one_model.csv',index=False)
print(de)

            name   auc_roc
0  all_algorithm  0.944522
1   remove_dmpnn  0.941240
2      remove_lr  0.937330
3    remove_fcnn  0.943388
4     remove_xgb  0.940904
